In [14]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [29]:
df = pd.read_csv('train_data.csv')

In [30]:
print(train_data.shape)

(42000, 785)


In [31]:
def sampling(tdata,val):
    len1= int(len(tdata)*val)
    t1 = tdata.sample(frac=1)
    val_data = t1.iloc[0:len1,:]
    train = t1.iloc[len1:,:]
    return val_data,train

v,d = sampling(df,0.2)

In [33]:
print(v.shape)

(8400, 785)


In [55]:
def generator(train,batch_size):
    count = 0
    while count<len(train):
        data = train.iloc[count:count+batch_size,:]
        X = data.iloc[:,1:]
        Y = data.iloc[:,0]
        yield X.values,Y.values
        count+=batch_size
    
    
gen= generator(d,8)


In [56]:
print(next(gen))

(array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64), array([9, 3, 8, 3, 8, 4, 4, 4], dtype=int64))


In [57]:
for i in range(5):
    v,d = sampling(df,0.2)
    generator1 = generator(d,64)
    
    for i, (x,y) in enumerate(generator1):
        print(x.shape)
        print(y.shape)
    print(i)

(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784

(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784

(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784

(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784

(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784)
(64,)
(64, 784